In [17]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

url= 'https://scholar.google.com/scholar'

#set headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) \
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36' 
    #english
}

params = {
    #english
    'hl': 'en',

    'start': 0,
    'q': 'circular economy waste household' 
}


In [22]:
##GET TITLE/LINK/AUTHORS OF N*10 ARTICLES

#loop through pages for n pages
n = 10

from tqdm.notebook import tqdm

#store results
results = pd.DataFrame(columns = ['titles', 'links', 'authors'])

for i in tqdm(range(n)):
    params['start'] = i*10
    response = requests.get(url, headers=headers, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('div', class_='gs_ri')

    #get titles
    titles = [article.find('h3', class_='gs_rt').text for article in articles]
    #locate and strip the '[HTML]' or '[PDF]' in the titles
    titles = [title[:title.find('[')-1] if title.find('[') != -1 else title for title in titles]

    #get links
    links = [article.find('h3', class_='gs_rt').find('a')['href'] for article in articles]
    #get authors
    authors = [article.find('div', class_='gs_a').text for article in articles]
    
    #add to results
    results = results._append(pd.DataFrame({'titles' : titles, 'links' : links, 'authors' : authors}))


  0%|          | 0/10 [00:00<?, ?it/s]

In [23]:
results.to_csv('article_info.csv')

In [19]:
## BASIC CODE TO SCRAPE ABSTRACT 

import requests
from bs4 import BeautifulSoup

def scrape_abstract(url):

    # YOUR CODE HERE
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        abstract = soup.find('blockquote', class_='abstract').text.strip()
    
    return abstract

In [5]:
link = 'https://www.sciencedirect.com/science/article/pii/S0921800919302976'

response = requests.get(link, headers=headers)

print(response)

<Response [403]>


In [21]:
## CODE TO DOWNLOAD 1 LINK

from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import requests
from urllib.parse import urlparse

main_page_url = 'https://sci-hub.ru/'
url_to_search = 'https://www.sciencedirect.com/science/article/pii/S0921800919302976'
driver = webdriver.Chrome()

try:
    # Open page
    driver.get(main_page_url)
    time.sleep(2)  # Wait for the page to load (you can decrease this time if need be)

    #doi_input = driver.find_element()
    # Find search bar -> input URL
    search_bar = driver.find_element(by=By.ID, value='request')
    search_bar.send_keys(url_to_search)
    search_bar.send_keys(Keys.RETURN)  # To submit the search query
    time.sleep(1)  # Wait for a bit again

    # Find download link
    download_button = driver.find_element(by=By.XPATH, value="//button[@onclick]")
    onclick_attribute = download_button.get_attribute('onclick')

    # Extract URL from the onclick attribute
    # Assuming the format is location.href='URL'
    pdf_url = onclick_attribute.split("'")[1]
    
    download_link = 'https://' + urlparse(main_page_url).hostname + pdf_url
  
    # Download the PDF using requests
    response = requests.get(download_link)

    with open('pdf.pdf', 'wb') as file:
        file.write(response.content)

except Exception as e:
    # Handle the exception, print an error message, etc.
    print(f"An error occurred: {str(e)}")
finally:
    driver.quit()

In [4]:
#scraping function

from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import requests

def pdf_download(url_to_search, file_name):
    driver = webdriver.Chrome() #initialize web-finder
    main_page_url = 'https://sci-hub.ru/' #open sci-hub
    driver.get(main_page_url)
    time.sleep(1)
    # Find search bar -> input URL
    search_bar = driver.find_element(by=By.ID, value='request')
    search_bar.send_keys(url_to_search)
    search_bar.send_keys(Keys.RETURN)  # To submit the search query
    time.sleep(1)
    
    try:
        # Find download link
        download_button = driver.find_element(by=By.XPATH, value="//button[@onclick]")
        onclick_attribute = download_button.get_attribute('onclick')
    except:
        driver.quit()
        return url_to_search

    
    # Extract URL from the onclick attribute
    # Assuming the format is location.href='URL'
    pdf_url = onclick_attribute.split("'")[1]
    download_link = 'https://sci-hub.ru' + pdf_url
    # Download the PDF using requests
    response = requests.get(download_link)
    with open(f'pdfs/{file_name}.pdf', 'wb') as file:
        file.write(response.content)
    
    driver.quit()

    return None



In [36]:
##to call on all links !!! DO NOT RERUN, IT WILL SET LIST BACK TO NONE

import pandas as pd
import os
from tqdm.notebook import tqdm

data = pd.read_csv('article_info.csv')
title = data['title']
links = data['link']
author = ['author'] 


def file_exists(file_path):
    return os.path.exists(file_path)

list_error_links = []


In [40]:
#function to call on all links

for index, row in data.iterrows():
    pdf_file_path = f'pdfs/{index}.pdf'
    if file_exists(pdf_file_path) is True:
        print("PDF already downloaded")
    else:
        if row['link'] in list_error_links:
            print('error already encountered')
        else:
            url_returned = pdf_download(row['link'], index)  #(links[index], index)
            if url_returned is not None:
                list_error_links.append(url_returned)
                print(f'ERROR FOR {url_returned}')

PDF already downloaded
error already encountered
error already encountered
PDF already downloaded
PDF already downloaded
error already encountered
error already encountered
error already encountered
PDF already downloaded
PDF already downloaded
error already encountered
PDF already downloaded
error already encountered
PDF already downloaded
PDF already downloaded
PDF already downloaded
PDF already downloaded
PDF already downloaded
error already encountered
PDF already downloaded
PDF already downloaded
error already encountered
error already encountered
PDF already downloaded
PDF already downloaded
error already encountered
error already encountered
PDF already downloaded
ERROR FOR https://www.sciencedirect.com/science/article/pii/S0892687523001681
ERROR FOR https://www.tandfonline.com/doi/abs/10.1080/10962247.2016.1229700


Exception ignored in: <function Service.__del__ at 0x0000027F4CB417E0>
Traceback (most recent call last):
  File "c:\Users\celia\AppData\Local\Programs\Python\Python310\lib\site-packages\selenium\webdriver\common\service.py", line 185, in __del__
    self.stop()
  File "c:\Users\celia\AppData\Local\Programs\Python\Python310\lib\site-packages\selenium\webdriver\common\service.py", line 146, in stop
    self.send_remote_shutdown_command()
  File "c:\Users\celia\AppData\Local\Programs\Python\Python310\lib\site-packages\selenium\webdriver\common\service.py", line 126, in send_remote_shutdown_command
    request.urlopen(f"{self.service_url}/shutdown")
  File "c:\Users\celia\AppData\Local\Programs\Python\Python310\lib\urllib\request.py", line 216, in urlopen
    return opener.open(url, data, timeout)
  File "c:\Users\celia\AppData\Local\Programs\Python\Python310\lib\urllib\request.py", line 519, in open
    response = self._open(req, data)
  File "c:\Users\celia\AppData\Local\Programs\Python

ERROR FOR https://www.sciencedirect.com/science/article/pii/S0959652620340920
ERROR FOR https://www.mdpi.com/1996-1073/14/19/6164


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.199)
Stacktrace:
	GetHandleVerifier [0x00007FF6B27182B2+55298]
	(No symbol) [0x00007FF6B2685E02]
	(No symbol) [0x00007FF6B25405AB]
	(No symbol) [0x00007FF6B2520038]
	(No symbol) [0x00007FF6B25A6BC7]
	(No symbol) [0x00007FF6B25BA15F]
	(No symbol) [0x00007FF6B25A1E83]
	(No symbol) [0x00007FF6B257670A]
	(No symbol) [0x00007FF6B2577964]
	GetHandleVerifier [0x00007FF6B2A90AAB+3694587]
	GetHandleVerifier [0x00007FF6B2AE728E+4048862]
	GetHandleVerifier [0x00007FF6B2ADF173+4015811]
	GetHandleVerifier [0x00007FF6B27B47D6+695590]
	(No symbol) [0x00007FF6B2690CE8]
	(No symbol) [0x00007FF6B268CF34]
	(No symbol) [0x00007FF6B268D062]
	(No symbol) [0x00007FF6B267D3A3]
	BaseThreadInitThunk [0x00007FF9C041257D+29]
	RtlUserThreadStart [0x00007FF9C15EAA58+40]


Scrape Discussion

In [53]:
%pip install pypdf

   ---------------------------------------- 0.0/277.6 kB ? eta -:--:--
   ---------------------------------------- 277.6/277.6 kB 8.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
from pypdf import PdfReader
from os import listdir
from os.path import isfile, join
import os
from tqdm.notebook import tqdm

mypath = os.getcwd() + '\\pdfs'


pdf_folder_content = [f for f in listdir(mypath) if isfile(join(mypath, f))]

all_pdf_texts = []

for pdf in pdf_folder_content:
    print(f'Analyzing {pdf}...')
	
    reader = PdfReader(f'{mypath}/{pdf}')

    n_pages = len(reader.pages)
	
    print(f'{pdf} has {n_pages} pages!')

    all_text = []

    for i in range(0,n_pages):  
      page_text = reader.pages[i].extract_text()
      all_text.append(page_text)

    result_string = ' '.join(all_text)
    all_pdf_texts.append(result_string)

##SWITCH TO A DICTIONARY INSTEAD OF A LIST

Analyzing 0.pdf...
0.pdf has 12 pages!
Analyzing 11.pdf...
11.pdf has 12 pages!
Analyzing 14.pdf...
14.pdf has 11 pages!
Analyzing 15.pdf...
15.pdf has 6 pages!
Analyzing 16.pdf...
16.pdf has 11 pages!
Analyzing 17.pdf...
17.pdf has 8 pages!
Analyzing 19.pdf...
19.pdf has 18 pages!
Analyzing 23.pdf...
23.pdf has 36 pages!
Analyzing 24.pdf...
24.pdf has 16 pages!
Analyzing 27.pdf...
27.pdf has 19 pages!
Analyzing 3.pdf...
3.pdf has 9 pages!
Analyzing 8.pdf...
8.pdf has 14 pages!


In [71]:
print(all_pdf_texts)
text = all_pdf_texts[0].split(' ')

# replace \n with space, then split it by the space
text = [word.replace('\n', ' ') if '\n' in word else word for word in text]
text = ' '.join(text).split(' ')

count = 0

# split, then join
# filter after tokenise then filter by list comprehension
#l for l in list, if l = discussioin etc


# delete all elements before the first one that contains the word "discussion"
for word in text:
    if 'Discussion' in word:
        text = text[count + 1:]
        break
    
    if 'Conclusion' in word:
        text = text[:count]
        break
    
    count += 1

text = ' '.join(text).split('. ')

print(text)

["Contents lists available at ScienceDirect\nEcological Economics\njournal homepage: www.elsevier.com/locate/ecolecon\nAnalysis\nConsumers in a Circular Economy: Economic Analysis of Household Waste\nSorting Behaviour\nDoan Nainggolana,⁎, Anders Branth Pedersena, Sinne Smedb, Kahsay Haile Zemoa, Berit Haslera,\nMette Termansenb\naDepartment ofEnvironmental Science, AarhusUniversity, Frederiksborgvej 399, DK 4000 Roskilde, Denmark\nbDepartment ofFood and Resource Economics, CopenhagenUniversity, Rolighedsvej 23, DK-1958 Frederiksberg C., Denmark\nARTICLE INFO\nKeywords:\nCircular economy\nWaste sorting\nRecycling\nDiscrete choice experimentChoice model\nHousehold preference\nSurvey\nSegmentation\nLatent class modelABSTRACT\nThepresentresearchprovidesaquantitativeassessmentofhouseholds'preferencesfordifferentwasteseparation\nandhandlingschemes.Weemployachoice-experimentbased,nation-widesurveyacrossDenmark.Thefindingssupport four different types of “waste sorters”; reflecting the heteroge

In [84]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import re


from nltk.stem import WordNetLemmatizer
from gensim import corpora
from gensim.models.ldamodel import LdaModel

lemmatizer = WordNetLemmatizer()

additional_stopwords = ['et', 'al','b', 'fw', 'e'] 

def preprocess_text(text):
    #YOUR CODE HERE

    # removes special characters and punctuation (STEP 1: Text cleaning)
    text = re.sub(r'[^A-Za-z\s]', '', text) 
    # remove additional white spaces (STEP 1: Text cleaning)
    text = re.sub(r'\s+', ' ', text).strip() 

    #put text in lowercases (STEP 2: Case normalization)
    text = text.lower()

    #STEP 3: tokenization
    tokens = word_tokenize(text)

    #STEP 4: removing stopwords:
    stopwords_list = set(stopwords.words('english')).union(additional_stopwords)
    tokens = [token for token in tokens if token not in stopwords_list]

    #STEP 5: lemmeatization: 
    lemmatized_words = [lemmatizer.lemmatize(token) for token in tokens]
    
    return lemmatized_words

In [92]:
processed_text = [preprocess_text(text) for text in all_pdf_texts]

corpus = corpora.Dictionary(processed_text)
doc_term_matrix = [corpus.doc2bow(text) for text in processed_text]

num_topics = 45

lda_model = LdaModel(doc_term_matrix, id2word = corpus, num_topics = num_topics)



In [93]:
from collections import Counter

document_topics = [lda_model.get_document_topics(doc) for doc in doc_term_matrix]
all_topics = [topic[0] for topics in document_topics for topic in topics]

topics_nb = Counter(all_topics)

most_nb_topics = [topic for topic, count in topics_nb.most_common()]
print("The most common topics and the number of instances are:")
for topic, x in topics_nb.most_common():
    print(f"Topic {topic +1}: Count {x}")
for topic in most_nb_topics:
    topics_words = lda_model.show_topic(topic)
    most_words = [word for word, _ in topics_words]
    print(f"Most used words in Topic {topic +1}: {', '.join(most_words)}")

The most common topics and the number of instances are:
Topic 16: Count 6
Topic 44: Count 4
Topic 9: Count 3
Topic 29: Count 3
Topic 2: Count 2
Topic 43: Count 2
Topic 31: Count 1
Topic 21: Count 1
Topic 38: Count 1
Topic 6: Count 1
Topic 10: Count 1
Topic 5: Count 1
Most used words in Topic 16: waste, plastic, household, recycling, food, economy, product, collection, system, material
Most used words in Topic 44: waste, recycling, household, plastic, collection, product, ewaste, study, system, business
Most used words in Topic 9: waste, plastic, recycling, material, business, household, food, study, packaging, korea
Most used words in Topic 29: waste, household, food, recycling, consumer, product, plastic, collection, study, material
Most used words in Topic 2: waste, food, product, recycling, consumer, ewaste, household, study, collection, system
Most used words in Topic 43: waste, recycling, system, product, ewaste, household, study, consumer, food, management
Most used words in Topi

In [91]:
from gensim.models.coherencemodel import CoherenceModel

# Function to compute coherence score
def compute_coherence(dictionary, corpus, texts, limit, start=2, step=3):
    # YOUR CODE HERE
    coherence_values = []
    for num_topics in range(start, limit, step):
        lda_model = LdaModel(corpus = corpus, id2word = dictionary, num_topics = num_topics, random_state = 100)

        coherence_model_lda = CoherenceModel(model = lda_model, texts = texts, dictionary = dictionary, coherence = 'c_v')
        coherence_values.append(coherence_model_lda.get_coherence())
    
    return coherence_values

coherence_values = compute_coherence(dictionary=corpus, corpus= doc_term_matrix, texts=processed_text, limit=50, start = 5, step=5)

optimal_nb_topics = (coherence_values.index(max(coherence_values)) + 1) * 5
print(f"the optimal number of topic for this model is: {optimal_nb_topics}")

the optimal number of topic for this model is: 45
